# Com dimensão

In [28]:
%%time
from ortools.sat.python import cp_model
import pandas

#---------------------------------------------------
# data 
#---------------------------------------------------

data = {'bin':{'h':60,'w':40},
        'cat1':{'w': 7,'h':12,'items':10},
        'cat2':{'w': 9,'h': 3,'items':10},
        'cat3':{'w': 5,'h':14,'items':10},
        'cat4':{'w':13,'h': 9,'items':10},
        'cat5':{'w': 6,'h': 8,'items': 5},
        'cat6':{'w':20,'h': 5,'items': 5}}      

#
# extract data for easier access
#

# bin width and height
H = data['bin']['h']
W = data['bin']['w']

# h,w,cat for each item
h = [data[cat]['h'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
w = [data[cat]['w'] for cat in data if cat!='bin' for i in range(data[cat]['items'])]
cat = [cat for cat in data if cat!='bin' for i in range(data[cat]['items'])]
n = len(h)  # number of items
m = 10  # max number of bins

#---------------------------------------------------
# or-tools model 
#---------------------------------------------------

model = cp_model.CpModel()

#
# variables
#

# x and y
x = [model.NewIntVar(0,W-w[i],f'x{i}') for i in range(n)]

xb1 = [model.NewIntVar(0,m*W-w[i],f'xb1.{i}') for i in range(n)]
xb2 = [model.NewIntVar(w[i],m*W,f'xb2.{i}') for i in range(n)]

y1 = [model.NewIntVar(0,H-h[i],f'y1.{i}') for i in range(n)]
y2 = [model.NewIntVar(h[i],H,f'y2.{i}') for i in range(n)]

# interval variables
xival = [model.NewIntervalVar(xb1[i],w[i],xb2[i],f'xival{i}') for i in range(n)]
yival = [model.NewIntervalVar(y1[i],h[i],y2[i],f'yival{i}') for i in range(n)]

# bin numbers
b = [model.NewIntVar(0,m-1,f'b{i}') for i in range(n)]

# objective
z = model.NewIntVar(0,m-1,'z')

#
# constraints
#
for i in range(n):
  model.Add(xb1[i] == x[i] + b[i]*W)
  model.Add(xb2[i] == xb1[i] + w[i])

model.AddNoOverlap2D(xival,yival)

model.AddMaxEquality(z,[b[i] for i in range(n)])

# objective
model.Minimize(z)    

#
# solve model
#
solver = cp_model.CpSolver()

# log does not work inside a Jupyter notebook
solver.parameters.log_search_progress = True
rc = solver.Solve(model)
print(f"return code:{rc}")
print(f"status:{solver.StatusName()}")

if rc == 4:
    df = pandas.DataFrame({ 
        'bin' : [solver.Value(b[i]) for i in range(n)],
        'x'   : [solver.Value(x[i]) for i in range(n)],
        'y'   : [solver.Value(y1[i]) for i in range(n)],
        'w'   : w,
        'h'   : h})
    display(df)


Starting CP-SAT solver v9.7.2996
Parameters: log_search_progress: true
Setting number of workers to 8

Initial optimization model '': (model_fingerprint: 0x939297ecad356fad)
#Variables: 301 (#ints: 1 in objective)
  - 51 in [0,9]
  - 5 in [0,20]
  - 10 in [0,27]
  - 10 in [0,31]
  - 10 in [0,33]
  - 5 in [0,34]
  - 10 in [0,35]
  - 10 in [0,46]
  - 10 in [0,48]
  - 10 in [0,51]
  - 5 in [0,52]
  - 5 in [0,55]
  - 10 in [0,57]
  - 5 in [0,380]
  - 10 in [0,387]
  - 10 in [0,391]
  - 10 in [0,393]
  - 5 in [0,394]
  - 10 in [0,395]
  - 10 in [3,60]
  - 5 in [5,60]
  - 10 in [5,400]
  - 5 in [6,400]
  - 10 in [7,400]
  - 5 in [8,60]
  - 10 in [9,60]
  - 10 in [9,400]
  - 10 in [12,60]
  - 10 in [13,400]
  - 10 in [14,60]
  - 5 in [20,400]
#kInterval: 100
#kLinMax: 1 (#expressions: 50)
#kLinear2: 150
#kLinear3: 50
#kNoOverlap2D: 1 (#rectangles: 50)

Starting presolve at 0.00s
[ExtractEncodingFromLinear] #potential_supersets=0 #potential_subsets=0 #at_most_one_encodings=0 #exactly_one_enco

CpSolverResponse summary:
status: OPTIMAL
objective: 1
best_bound: 1
integers: 151
booleans: 506
conflicts: 2359
branches: 7639
propagations: 52166
integer_propagations: 25346
restarts: 8
lp_iterations: 0
walltime: 0.337622
usertime: 0.337622
deterministic_time: 0.099448
gap_integral: 0.0330268
solution_fingerprint: 0x93acf130d7a367bf



In [29]:
df

,bin,x,y,w,h
0,0,11,0,7,12
1,0,0,39,7,12
2,0,13,29,7,12
3,0,29,43,7,12
4,1,28,0,7,12
5,0,33,31,7,12
6,1,5,9,7,12
7,1,12,12,7,12
8,1,5,21,7,12
9,1,26,12,7,12
